# Testing notebook

In [1]:
import xlsxwriter
import pandas as pd
import numpy as np

from classification_power_predictor import classification_power_predictor
from writing import *

ModuleNotFoundError: No module named 'DS_common.excel_writing'

## We need some testing data

In [ ]:
# frame size
n = 1000

test_frame = pd.DataFrame({"numeric_variable": np.random.normal(10, 5, n), 
                           "object_variable": np.round(np.random.uniform(0, 10, n)).astype('O'),
                           "column with soo000oo000oo000oo000oo long name": np.random.normal(10, 5, n)})

nv = test_frame["numeric_variable"]
ov = test_frame["object_variable"]

probs1 = nv.apply(lambda x: (x/(max(nv) - min(nv))) + ((np.random.rand())/5))
probs2 = ov.apply(lambda x: (x/(max(ov) - min(ov))) + ((np.random.rand())/5))

f_probs = (probs1 + probs2)/2
f_probs[f_probs >= 1] = 1


Y = np.zeros(n)
Y[f_probs > 0.4] = 1
Y[f_probs > 0.7] = 2

Y = pd.Series(Y)
Y_strs = Y.replace({0: 'cetegory 0', 1:'category 1', 2:'category 2'})
Y.value_counts()
Y_binary = Y.replace({2:1})

na_containts_frame = test_frame.copy()

for col in test_frame.columns:
    na_containts_frame.loc[na_containts_frame.sample(int(n/50)).index, col] = np.NaN

## Computions funcitons

In [ ]:
from computions import *

### get_describe_nominal function test

Basic situation

In [ ]:
get_describe_nominal(test_frame.iloc[:,1], Y)

,count,0.0,0.0%,1.0,1.0%,2.0,2.0%
object_variable,,,,,,,
5.0,117,15.0,12.820513,101,86.324786,1.0,0.854701
6.0,112,9.0,8.035714,97,86.607143,6.0,5.357143
2.0,103,79.0,76.699029,24,23.300971,0.0,0.000000
8.0,100,0.0,0.000000,70,70.000000,30.0,30.000000
9.0,94,0.0,0.000000,51,54.255319,43.0,45.744681
3.0,94,50.0,53.191489,44,46.808511,0.0,0.000000
4.0,91,30.0,32.967033,61,67.032967,0.0,0.000000
7.0,91,0.0,0.000000,80,87.912088,11.0,12.087912
1.0,90,76.0,84.444444,14,15.555556,0.0,0.000000


Data frame with emptys

In [ ]:
test = get_describe_nominal(na_containts_frame.iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))
test = get_describe_nominal(na_containts_frame.fillna('empty').iloc[:,1], Y)
print("Is all obs. are in table - ",  str(test['count'].sum() == n))

Is all obs. are in table -  False
Is all obs. are in table -  True


### AUC computing test

AUC numeric - simple fucntion

In [ ]:
print(get_AUC_numeric(test_frame['numeric_variable'] ,Y))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(na_containts_frame['numeric_variable'], Y_strs))
print(get_AUC_numeric(test_frame['numeric_variable'], Y_binary))

{0.0: 0.29587826871474343, 1.0: 0.5497199333020351, 2.0: 0.7741065466434253}
{'cetegory 0': 0.29587826871474343, 'category 1': 0.5497199333020351, 'category 2': 0.7741065466434253}
{'cetegory 0': 0.2954687013391467, 'category 1': 0.5494883720930233, 'category 2': 0.7818482584290548}
{0.0: 0.29587826871474343, 1.0: 0.7041217312852567}


AUC nominal

In [ ]:
non_table = get_AUC_nominal(test_frame['object_variable'], Y)
dn_tab = get_describe_nominal(test_frame.iloc[:,1], Y_strs)
with_table = get_AUC_nominal(test_frame['object_variable'], Y_strs, dn_tab)
print(with_table)

print('AUC bynary')
print(get_AUC_nominal(test_frame['object_variable'], Y_binary))

print('AUC with emptys - needs to be replaces in previous steps')
print(get_AUC_nominal(na_containts_frame['object_variable'].fillna('empty'), Y_strs))

get_AUC_nominal(na_containts_frame['object_variable'].fillna('empty'), Y_strs, 
                get_describe_nominal(na_containts_frame.iloc[:,1].fillna('empty'), Y_strs))


{'cetegory 0': 0.9283940455097172, 'category 1': 0.8249987858380147, 'category 2': 0.9276348310221921}
AUC bynary
{0.0: 0.9283940455097172, 1.0: 0.9283940455097173}
AUC with emptys - needs to be replaces in previous steps
{'cetegory 0': 0.925687459017537, 'category 1': 0.8213724887082935, 'category 2': 0.9239838350200761}


{'cetegory 0': 0.925687459017537,
 'category 1': 0.8213724887082935,
 'category 2': 0.9239838350200761}

Get publish AUC for any predictor type

In [ ]:
non_table = get_full_AUC(test_frame['object_variable'], Y, 'nominal')
print('nominal predictor without table')
print(non_table)

with_table = get_full_AUC(test_frame['object_variable'], Y_strs, 'nominal', descr_table = dn_tab)
print('moninal predictor with table')
print(with_table)

numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
print('numeric predictor')
print(numeric_var)

nominal predictor without table
{0.0: {'AUC': 0.9283940455097172, 'rel_type': 1, 'GINI': 0.8567880910194343}, 1.0: {'AUC': 0.8249987858380147, 'rel_type': 1, 'GINI': 0.6499975716760293}, 2.0: {'AUC': 0.9276348310221921, 'rel_type': 1, 'GINI': 0.8552696620443843}}
moninal predictor with table
{'cetegory 0': {'AUC': 0.9283940455097172, 'rel_type': 1, 'GINI': 0.8567880910194343}, 'category 1': {'AUC': 0.8249987858380147, 'rel_type': 1, 'GINI': 0.6499975716760293}, 'category 2': {'AUC': 0.9276348310221921, 'rel_type': 1, 'GINI': 0.8552696620443843}}
numeric predictor
{0.0: {'AUC': 0.7041217312852566, 'rel_type': -1, 'GINI': 0.40824346257051314}, 1.0: {'AUC': 0.5497199333020351, 'rel_type': 1, 'GINI': 0.09943986660407012}, 2.0: {'AUC': 0.7741065466434253, 'rel_type': 1, 'GINI': 0.5482130932868505}}


Decoding AUC info as DataFrame

In [ ]:
numeric_var = get_full_AUC(test_frame['numeric_variable'], Y, 'numeric')
AUC_info_to_DataFrame(numeric_var,'numeric_variable')

0.0                         1.0                    \
                       AUC rel_type      GINI      AUC rel_type     GINI   
numeric_variable  0.704122       -1  0.408243  0.54972        1  0.09944   

                       2.0                     
                       AUC rel_type      GINI  
numeric_variable  0.774107        1  0.548213

get_all_computions funcion test

In [ ]:
print('describe_table with no nas')
print(get_all_comuptions(test_frame['object_variable'], Y)['describe_table'])
print('describe_table with nas')
print(get_all_comuptions(na_containts_frame['object_variable'], Y)['describe_table'])

describe_table with no nas
                 count   0.0       0.0%  1.0       1.0%   2.0       2.0%
object_variable                                                         
5.0                117  15.0  12.820513  101  86.324786   1.0   0.854701
6.0                112   9.0   8.035714   97  86.607143   6.0   5.357143
2.0                103  79.0  76.699029   24  23.300971   0.0   0.000000
8.0                100   0.0   0.000000   70  70.000000  30.0  30.000000
9.0                 94   0.0   0.000000   51  54.255319  43.0  45.744681
3.0                 94  50.0  53.191489   44  46.808511   0.0   0.000000
4.0                 91  30.0  32.967033   61  67.032967   0.0   0.000000
7.0                 91   0.0   0.000000   80  87.912088  11.0  12.087912
1.0                 90  76.0  84.444444   14  15.555556   0.0   0.000000
0.0                 57  54.0  94.736842    3   5.263158   0.0   0.000000
10.0                51   0.0   0.000000    9  17.647059  42.0  82.352941
describe_table with nas


Getting indicators from computions uotput

In [ ]:
comp_result =  get_all_comuptions(test_frame['numeric_variable'], Y)
print('full numeric data')
get_predictor_row(comp_result)
print('numeric data with nas, with str categoryes')
comp_result =  get_all_comuptions(na_containts_frame['numeric_variable'], Y_strs)
get_predictor_row(comp_result)

full numeric data
numeric data with nas, with str categoryes


cetegory 0                    category 1                     \
                        AUC rel_type      GINI        AUC rel_type      GINI   
numeric_variable   0.704531       -1  0.409063   0.549488        1  0.098977   

                 category 2                    Emptys Emptys part  
                        AUC rel_type      GINI                     
numeric_variable   0.781848        1  0.563697     20        0.02

## Writing to excel

Adding a info about different predictors

In [ ]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()

Let's show final table

In [ ]:
my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
#my_cpp.get_predictors_data()
my_cpp.result_DF

0.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.704122       -1  0.408243   
object_variable                                0.928394        1  0.856788   
column with soo000oo000oo000oo000oo long name  0.500937       -1  0.001874   

                                                    1.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                                0.54972        1   0.09944   
object_variable                                0.824999        1  0.649998   
column with soo000oo000oo000oo000oo long name  0.512635       -1  0.025271   

                                                    2.0                     \
                                                    AUC rel_type      GINI   
numeric_variable                               0.774107        1  0.548213   
object_variable                                0.927635        1   0.85527   
column with soo000oo000oo000oo000oo long name  0.528822        1  0.057644   

                                              Emptys Emptys part  
                                                                  
numeric_variable                                   0         0.0  
object_variable                                    0         0.0  
column with soo000oo000oo000oo000oo long name      0         0.0

Writig a double header table

In [ ]:


xl_writer = pd.ExcelWriter("test_result/double_header_saver.xlsx",engine='xlsxwriter')
ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws      

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

print_double_column_header(  my_cpp, ws, 
                            my_cpp.result_DF.loc[['numeric_variable'],:],
                            "A1", 'Hello World', 
                            xl_writer.book.add_format(my_cpp.default_header_format))

xl_writer.close()

Writing info about some predictor

In [ ]:
xl_writer = pd.ExcelWriter("test_result/writing_a_value.xlsx",engine='xlsxwriter')

ws = xl_writer.book.add_worksheet('test')
xl_writer.sheets['test'] = ws                             

my_cpp = classification_power_predictor(test_frame, Y)
my_cpp.update_predictors()
my_cpp.my_writer = xl_writer

default_predictor_printer(my_cpp, ws, my_cpp.get_predictors_data()['numeric_variable'], 
{'header_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'desc_format':xl_writer.book.add_format(my_cpp.default_header_format),
  'class_ab_format':xl_writer.book.add_format(my_cpp.default_header_format)})

xl_writer.close()

In [ ]:
xl_writer = pd.ExcelWriter("test_result/result_test.xlsx",engine='xlsxwriter')
my_cpp.write_to_book(xl_writer)
xl_writer.close()